# >Título atractivo

## <span style="color:rgb(204, 153, 0)"> Descripción del juego Clash of Clans</span>
También conocido como CoC, Clash of Clans es un videojuego de estrategia multijugador en el que los jugadores asumen el papel de jefes de clanes y construyen y mejoran sus aldeas. Al mismo tiempo, el usuario puede crear o unirse a un clan, donde podrá interactuar con otros jugadores.

El juego se desarrolla en un entorno multijugador en línea, lo que permite a los jugadores interactuar con otros jugadores de todo el mundo a través de chat y sistema de clanes.

Fue desarrollado por Supercell y lanzado para dispositivos iOS el 2 de agosto del 2012 y el 8 de octubre para dispositivos Android.

Antes de iniciar con el WebScraping, necesitamos crear una cuenta en la web API de Clash of Clans (https://developer.clashofclans.com/#/register) para obtener nuestra clave token.

<img src="https://i.pinimg.com/originals/11/ae/b7/11aeb7ddb3f1d518648a1e36b32b2069.png" alt="Clash of Clans" width="500">

## <span style="color: #34253B"> WebScraping mediante API's [ Clash of Clans ] </span>

## <span style="color: #597aa2"> Conexión con la url (str) </span>


In [2]:
#Importamos las librerías que vamos a necesitar en nuestro trabajo
import urllib.request
from urllib.request import urlopen
import urllib.parse
import pandas as pd
import requests
import json
import polars as pl
import re

#Abrimos el archivo "key.txt" y leemos el token generado guardado en un archivo texto
with open("keyJimena.txt") as f:
    my_key2 = f.read().rstrip("\n")

#URL base de la API de Clash of Clans
base_url = "https://api.clashofclans.com/v1"

#Parámetro para obtener todos los clanes que tengan 20 o más miembros
endpoint = "/clans?minMembers=20"

#Creamos una solicitud HTTP con la clave de autenticación en el encabezado
request = urllib.request.Request(
    base_url + endpoint,
    None,
    {"Authorization": "Bearer %s" % my_key2}
)

# Hacemos la solicitud y obtenemos la respuesta
uclient = urlopen(request)
responsehtml = uclient.read()
uclient.close()

# Decodificamos la respuesta en formato JSON
datos = responsehtml.decode()

# Tenemos los datos obtenidos
#print(datos)

### <span style="color: #C340D6"> Pasar a diccionario (dict) </span>


In [3]:
#Convertimos a diccionario con json ya que esta Api está en este formato
datos_dict = json.loads(datos)
print(type(datos_dict))

<class 'dict'>


### <span style="color: #A6BD4C"> Primer filtrado (df) </span>

In [4]:
#Vamos a filtrar tales aspectos y agruparlos de acuerdo a la cantidad de puntos de los clanes, 
#con el fin de compararlo con Clash Royale

puntos_clanes = []
nombres_clanes = []
tipos = []
location = []
niveles_clanes = []
miembros = []
tag = []

for item in datos_dict["items"]:
    puntos_clanes.append(item['clanPoints'])
    nombres_clanes.append(item['name'])
    tipos.append(item['type'])
    ubicacion = item['location']['name'] if 'location' in item and 'name' in item['location'] else ''
    location.append(ubicacion)
    niveles_clanes.append(item['clanLevel'])
    miembros.append(item['members'])
    tag.append(item['tag'])

#Creamos el diccionario
dff = {
    'ClanPoints CoC': puntos_clanes,
    'Nombre CoC': nombres_clanes,
    'Tipo CoC': tipos,
    'Location CoC': location,
    'Nivel de clan CoC': niveles_clanes,
    'Miembros CoC': miembros,
    'Tag CoC': tag}

### <span style="color: #F34230"> Pasar a dataframe (df) </span>

In [5]:
#Creamos el dataframe con Pandas
DFF = pd.DataFrame(dff)
#print(DFF)

DFF1 = DFF.sort_values(by='ClanPoints CoC', ascending=False).head(10).reset_index(drop=True)
#print(DFF)

#Ya que la información obtenida es en tiempo real, esto nos ayuda a fijar siempre el primer Tag y así usarlo posteriormente
#en operaciones relacionadas con dicho clan seleccionado.
primero = DFF1.loc[0, 'Tag CoC']
print(primero)

DFF1

#Observamos el clan con la mayoría de puntos:

#80LPLRYV


,ClanPoints CoC,Nombre CoC,Tipo CoC,Location CoC,Nivel de clan CoC,Miembros CoC,Tag CoC
0,37153,HnTV,inviteOnly,International,26,31,#80LPLRYV
1,35004,indian boys,inviteOnly,India,21,43,#8JRCPPY0
2,32309,SIANTURIURUNGNA,inviteOnly,Indonesia,10,32,#CJQQJ2U9
3,32255,正义联盟,inviteOnly,Malaysia,14,40,#89PV2VYC
4,32133,İMPOSSABLE,inviteOnly,Türkiye,18,35,#28GV88J8
5,31880,TEAM BROTHERS,open,India,14,50,#2YVLUPRJJ
6,31661,SpArTaN,inviteOnly,Iran,16,37,#YLUUR9PV
7,31385,Warriors on way,inviteOnly,International,16,47,#JPGUJG8Q
8,30984,venomous,inviteOnly,Åland Islands,20,31,#P8U9UUL0
9,30271,POWER BOX 2™️,inviteOnly,Germany,11,44,#2Y9RVYGQJ


<img src="https://pbs.twimg.com/media/FNUIgMjWYAEGc7t.jpg" alt="Clash of Clans" width="600">

## <span style="color: #597aa2"> Segunda conexión con la url (str) </span>

In [6]:
# Codificar el valor de "primero"
tags = urllib.parse.quote(primero)

# Leer la clave de autenticación desde el archivo "keyJimena.txt"
with open("keyJimena.txt") as f:
    my_key2 = f.read().rstrip("\n")

# URL base de la API de Clash of Clans
base_url = "https://api.clashofclans.com/v1"

# Construir el endpoint de la solicitud con el valor codificado
endpoint = "/clans/%s/members?minMembers=20" % tags

# Construir la solicitud con la clave de autenticación
request = urllib.request.Request(
    base_url + endpoint,
    None,
    {
        "Authorization": "Bearer %s" % my_key2
    }
)

# Realizar la solicitud y obtener la respuesta
uclient = urllib.request.urlopen(request)
responsehtml = uclient.read()
uclient.close()

# Decodificar los datos de la respuesta
datos1 = responsehtml.decode()
#print(datos1)

### <span style="color: #C340D6"> Pasar a diccionario (dict) </span>

In [7]:
datos_d = json.loads(datos1)
print(type(datos_d))
#print(datos_d)

<class 'dict'>


### <span style="color: #A6BD4C"> Segundo filtrado (df) </span>

In [8]:
#Vamos a filtrar los jugadores del clan con mayor cantidad de puntos
#Se crean las listas vacías para almacenar los datos de los jugadores
nombre = []
rol = []
experiencia = []
trofeos = []
rango = []

#Se hace una iteración sobre los elementos de datos_d["items"] y extraemos los valores
for i in range(len(datos_d["items"])):
    nombre.append(datos_d["items"][i]['name'])
    rol.append(datos_d["items"][i]['role'])
    experiencia.append(datos_d["items"][i]['expLevel'])
    trofeos.append(datos_d["items"][i]['trophies'])
    rango.append(datos_d["items"][i]['clanRank'])

#Creamos un diccionario con las listas recopiladas
frame = {'Nombres CoC': nombre, 'Roles CoC': rol, 'Nivel de experiencia CoC': experiencia, 'Trofeos CoC': trofeos, 'Rango del clan CoC': rango}

### <span style="color: #F34230"> Pasar a dataframe (df) </span>

In [9]:
#Convertimos a dataframe
DF = pd.DataFrame(frame)

#Ordenamos en función a los trofeos obtenidos los miembros
DF1 = DF.sort_values(by='Trofeos CoC', ascending=False).head(10).reset_index(drop=True)

DF1

,Nombres CoC,Roles CoC,Nivel de experiencia CoC,Trofeos CoC,Rango del clan CoC
0,trantronghai,coLeader,257,5148,1
1,LTDungTediport,coLeader,257,5135,2
2,Mr Minh,coLeader,252,5131,3
3,an son,coLeader,218,5049,4
4,nhãn lồng,member,230,4972,5
5,Lê Hà_6688,coLeader,251,4957,6
6,TranTrongHai,coLeader,264,4932,7
7,Hec Met,coLeader,250,4908,8
8,hoanganhgialai,coLeader,243,4840,9
9,Trung Bất Tử,coLeader,247,4835,10


## <span style="color:rgb(75, 150, 171)"> Descripción del juego Clash Royale</span>


Tambien conocido como CR es un juego de estrategia y combate en tiempo real ambientada en el mismo universo de Clash of Clans, tambien desarrollada por Supercell. Combina elementos de juegos de cartas coleccionables, defensa de torres y estrategia multijugador en línea.
Antes de iniciar con el WebScraping necesitamos crear una cuenta en la web API https://developer.clashroyale.com/#/ para obtener nuestra clave token.


<img src="https://assets-prd.ignimgs.com/2022/05/27/clashroyale-1653673820137.jpg" alt="Clash of Clans" width="500">

## <span style="color: #34253B"> WebScraping mediante API's [ Clash Royale] </span>

## <span style="color: #709f1d"> Conexión con la url (str) </span>


In [1]:
import urllib.request
from urllib.request import urlopen
import pandas as pd
import requests
import json
import polars as pl
with open("my_keyArthur.txt") as f:
    my_key =f.read().rstrip("\n")
    
    base_url = "https://api.clashroyale.com/v1"
    
    #endpoint = "/cards"
    endpoint = "/clans?minScore=16000"
    
    request = urllib.request.Request(
        base_url+endpoint,
        None,
        {
            "Authorization": "Bearer %s" % my_key
        }
    )
    uclient=urlopen(request)
    responsehtml=uclient.read()
    #type(responsehtml)
    #uclient.close()
    #data = responsehtml.decode()

    #data
    #response = urllib.request.urlopen(request).read().decode("utf-8")
    
    data = responsehtml#.decode("utf-8")
    uclient.close()
    data
    #data = json.loads(response)
    #for item in data["items"]:
        #print("%s [%d]" % (item["name"], item["maxLevel"]))
        #d ="Clan: %s\nMembers: %d\nScore: %s\nTag: %s\n\n" % (item["name"], item["members"], item["clanScore"], item["tag"])
        #print(d)
   #print(data)

## <span style="color: #26a783"> Pasar a diccionario (dict) </span>

In [2]:
info= json.loads(data)
print(type(info))

<class 'dict'>


## <span style="color: #A6BD4C"> Filtrado (df) </span>

In [3]:
clanPoints = []
location=[]
member=[]
clan=[]
Type=[]
#print(info["items"][0])
for i in range(len(info["items"])):
    clanPoints.append(info["items"][i]['clanScore'])
    location.append(info["items"][i]['location']['name'])
    member.append(info["items"][i]["members"])
    clan.append(info ["items"][i]["name"])
    Type.append(info["items"][i]["type"])
dff={"Clan name (CR)":clan,'ClanPoints (CR)':clanPoints,"Type (CR)":Type,'Location (CR)':location,'Members (CR)':member}
#print(clanPoints)
#print(location)
DFF=pd.DataFrame(dff)

## <span style="color: #ab1ba5"> Pasar a dataframe (df) </span>

In [4]:
DFF=pd.DataFrame(dff)

#TOP 10 CLANES PARA LA FECHA 18/06/2023
DFF1 = DFF.sort_values(by='ClanPoints (CR)', ascending=False).head(10).reset_index(drop=True)

DFF1

,Clan name (CR),ClanPoints (CR),Type (CR),Location (CR),Members (CR)
0,Clan #LL8QPC8P,66464,open,United States,50
1,Clan #P9CL8R0G,64456,open,United States,50
2,kutingin,53851,closed,International,47
3,Return,52342,inviteOnly,Spain,50
4,喧嘩稼業,51003,closed,Japan,29
5,Hearthstone BR,49898,closed,Brazil,33
6,Acrifaal,45107,closed,Ukraine,21
7,om,43548,closed,International,24
8,금호동 본,43325,inviteOnly,South Korea,24
9,RoYaL BenGaL☢️,43038,inviteOnly,Bangladesh,20


<img src="https://i.ytimg.com/vi/biM_RUsfOso/maxresdefault.jpg" alt="Clash of Clans" width="500">

## <span style="color: #709f1d"> Segunda conexión con la url (str) </span>

In [11]:
with open("my_keyArthur.txt") as f:
    my_key =f.read().rstrip("\n")
    base_url = "https://api.clashroyale.com/v1"
    
    #endpoint = "/cards"
    endpoint = "/clans/%23LL8QPC8P/members"
    
    request = urllib.request.Request(
        base_url+endpoint,
        None,
        {
            "Authorization": "Bearer %s" % my_key
        }
    )
    uclient=urlopen(request)
    responsehtml=uclient.read()
    #type(responsehtml)
    #uclient.close()
    #data = responsehtml.decode()

    #data
    #response = urllib.request.urlopen(request).read().decode("utf-8")
    
    dataP = responsehtml#.decode("utf-8")
    uclient.close()
    dataP
    #data = json.loads(response)
    #for item in data["items"]:
        #print("%s [%d]" % (item["name"], item["maxLevel"]))
        #d ="Clan: %s\nMembers: %d\nScore: %s\nTag: %s\n\n" % (item["name"], item["members"], item["clanScore"], item["tag"])
        #print(d)
    #print(dataP)


## <span style="color: #26a783"> Pasar a diccionario (dict) </span>

In [12]:
infoP = json.loads(dataP)
print(type(infoP))


<class 'dict'>


## <span style="color: #A6BD4C"> Filtrado (df) </span>

In [14]:
nameP=[]
levelP=[]
rol=[]
trofeos=[]
rango=[]
for i in range(len(infoP["items"])):
    nameP.append(infoP["items"][i]["name"])
    levelP.append(infoP["items"][i]["expLevel"])
    rol.append(infoP["items"][i]["role"])
    trofeos.append(infoP["items"][i]["trophies"])
    rango.append(infoP["items"][i]["clanRank"])
dffP={'Player name':nameP,'Level':levelP,"role":rol,"Trophies":trofeos,"Clan Rank":rango}
DFFP=pd.DataFrame(dffP)

## <span style="color: #ab1ba5"> Pasar a dataframe (df) </span>

In [17]:
#TOP 10 CLANES PARA LA FECHA 18/06/2023
DFFP1 = DFFP.head(10)

DFFP1

,Player name,Level,role,Trophies,Clan Rank
0,Broads In ATL,45,elder,7305,1
1,TrapLord Truebs,50,coLeader,7030,2
2,Dicinbal,41,member,7000,3
3,MobyDck,50,leader,6771,4
4,Peter17$1.0,40,member,6733,5
5,kpopmattlauer,42,elder,6721,6
6,Imagine Losing,41,member,6712,7
7,Jeff,41,member,6709,8
8,William14K,50,member,6705,9
9,Rpooon14,42,member,6695,10



<img src="https://img.redbull.com/images/c_crop,w_1920,h_960,x_0,y_0,f_auto,q_auto/c_scale,w_1200/redbullcom/2020/3/13/hk3rfkvmjl4zx1d9wbze/clash-royale-temporada-9-meta" alt="Clash of Clans" width="500">

## <span style="color: #709f1d"> Tercera conexión con la url (str) </span>

In [5]:
with open("my_keyArthur.txt") as f:
    my_key =f.read().rstrip("\n")
    
    base_url = "https://api.clashroyale.com/v1"
    
    #endpoint = "/cards"
    endpoint = "/cards"
    
    request = urllib.request.Request(
        base_url+endpoint,
        None,
        {
            "Authorization": "Bearer %s" % my_key
        }
    )
    uclient=urlopen(request)
    responsehtml=uclient.read()
    #type(responsehtml)
    #uclient.close()
    #data = responsehtml.decode()

    #data
    #response = urllib.request.urlopen(request).read().decode("utf-8")
    
    data2 = responsehtml#.decode("utf-8")
    uclient.close()
    data2
    #data = json.loads(response)
    #for item in data["items"]:
        #print("%s [%d]" % (item["name"], item["maxLevel"]))
        #d ="Clan: %s\nMembers: %d\nScore: %s\nTag: %s\n\n" % (item["name"], item["members"], item["clanScore"], item["tag"])
        #print(d)
    #print(data2)

## <span style="color: #26a783"> Pasar a diccionario (dict) </span>

In [6]:
info2 = json.loads(data2)
print(type(info2))

<class 'dict'>


## <span style="color: #A6BD4C"> Filtrado (df) </span>

In [10]:
nombre = []
max_level=[]
ID=[]

for i in range(len(info2["items"])):
    nombre.append(info2["items"][i]['name'])
    max_level.append(info2["items"][i]['maxLevel'])
    ID.append(info2["items"][i]["id"])
    
dff2={'Nombre_carta':nombre,'Max_level':max_level,'ID':ID}
DFF_CARTAS=pd.DataFrame(dff2)



## <span style="color: #ab1ba5"> Pasar a dataframe (df) </span>

In [9]:
#TOP 10 CLANES PARA LA FECHA 18/06/2023
DFF_CARTAS = DFF_CARTAS.head(20)

DFF_CARTAS

,Nombre_carta,Max_level,ID
0,Knight,14,26000000
1,Archers,14,26000001
2,Goblins,14,26000002
3,Giant,12,26000003
4,P.E.K.K.A,9,26000004
5,Minions,14,26000005
6,Balloon,9,26000006
7,Witch,9,26000007
8,Barbarians,14,26000008
9,Golem,9,26000009


### >Merge a los DataFrame

### >Pasar a csv

### Errores
-


### >Conclusión